# Web3Copilot Tutorial

This notebook contains code snippets that will fill out missing components of the source code on the [`ethtoronto-tutorial` branch](https://github.com/chain-ml/web3-copilot/tree/ethtoronto-tutorial).

# Part 1: TransctionDebuggerSkill

## Init Skill

We're inheriting the `SkillBase` class, so let's give our skill a name at the top of the `__init__` method:

```python
        super().__init__(name="txn_debugger")
```

## Configure

### Imports
```python
from dotenv import find_dotenv, get_key
```

### Complete the `_get_config` method:
```python
    def _get_config(self):
        env_path = find_dotenv()

        web3_config = {
            "rpc_url": get_key(env_path, "ETH_MAINNET_URL"),
            "tenderly_api_key": get_key(env_path, "TENDERLY_API_KEY"),
            "block_explorer_api_key": get_key(env_path, "ETHERSCAN_API_KEY"),
            "etherscan_api": get_key(env_path, "ETHERSCAN_API"),
        }

        return web3_config
```

## Complete functions

### Extract prompt from context

In the `execute` method, assign the `query` variable as shown below:
```python
        query = context.chat_history.last_message.message
```

### Track Budget Consumptions

1. In the `execute` method, anywhere after the POST request that fetches thet transaction trace, add the following:
```python
        budget.add_consumption(Consumption(1, "call", "API_CALL"), self.name)
```

2. Do the same in the `fetch_contracts` method:
```python
        budget.add_consumption(Consumption(1, "call", "API_CALL"), self.name)
```

## Return skill result

At the end of `execute`, return this:
```python
        return self.build_success_message(
            f"{debug_context}",
            data=debug_context
        )
```

# Part 2: DocRetrievalSkill

## Init skill
Just as before, we need to give our skill a consistent name:

```python
        super().__init__(name="doc_retrieval")
```


## Complete the `execute` function

### Extract the prompt from `context`

```python
        query = context.chat_history.last_message.message
```

### Retrieve the documentation context

```python
        collection = self.db_client.get_or_create_collection(name=self.collection_name)
        doc_context = self.retriever.retrieve_docs(query=query, collection=collection)
```

### Return skill result

```python
        return self.build_success_message(
            f"Results from {self.collection_name} in database retrieved\n{doc_context}",
            data=doc_context
        )
```

# Part 3: Web3CopilotAgent

## Initialize skills


### Documentation Retrieval

1. As we will generate separate, unique PDFs for each web3 project, we should have a dedicated skill for them. These will be tracked by the `doc_retrieval_skills` dictionary. In the `init_skills` method, add the following code after the declaration of `doc_retrieval_skills`:

```python
        indices = list(create_file_dict(constants.DATA_DIR).keys())
        for index in indices:
            self.doc_retrieval_skills[index] = DocRetrievalSkill(
                db_client=self.db_client,
                collection_name=index,
                retriever=self.retriever,
            )
```

2. When we get our context message, we will have data that's unpresentable to users. Therefore we'll have to create another skill that converts the context messages from our main skill into plain English. That's where the `LLMSkill` class comes in. Right after the code above, create an LLM skill as follows:

```python
        self.dr_llm_skill = LLMSkill(
            llm=self.llm,
            system_prompt=self.load_system_prompt("doc_retrieval"),
            context_messages=self.build_context_message
        )
```

### Transaction Debugging

1. This skill is much simpler as it is very specific:
```python
        self.txn_debugger_skill = TransactionDebuggerSkill(
            config=self.config
        )
```
2. Create a corresponding LLMSkill for it:
```python
        self.txn_debugger_llm_skill = LLMSkill(
            llm=self.llm,
            system_prompt=self.load_system_prompt("txn_debugger"),
            context_messages=self.build_context_message
        )
```

## Initialize chains

Skills are contained in chains. The code has separate chains for each skill but the chain needs a `runner`. Let's populate that by adding the following snippets as parameters to the respective chains:

```python
                runners=[doc_retrieval_skill, self.dr_llm_skill],
```

```python
                runners=[self.txn_debugger_skill, self.txn_debugger_llm_skill],
```

## Templating: Prompts and Context Messages

LLMs take prompts as input. You would have noticed that our LLMSkill instances take in `system_prompt` and `context_messages` as arguments. The system prompt is passed as an instruction to your skill while the context message is used to construct the final message sent to the LLMSkill with your skill's response data included.

1. Birefly looking at the `load_system_prompt` method, you'll observe it references [Jinja2 template](https://jinja.palletsprojects.com/en/3.1.x/) files. Paste the following in each one:
```
{{ chat_history.user.last_message}}
```

2. In the `build_context_message` method, another template is being used. Carefully add the following where appropriate:
```
{{chain_history.last_message}}
```
```
{{chat_history.user.last_message}}
```

3. Council applies data onto the templates using its `PromptBuilder` class. To use this for your skills, update the `build_context_message` as follows:

```python
        context_message_prompt = PromptToMessages(
            prompt_builder=PromptBuilder(context_message_template)
        )

        messages = context_message_prompt.to_user_message(context)
        return messages
```


## Budgeting and Agent Execution

LLMs like OpenAI's GPT models have a cost to use. Council allows you to set limits on your agent's execution. Budgets are time-based by default but can take in other types of "consumptions" that you want to limit such as 3rd party API requests. Recall that in our `TransactionDebuggerSkill` class, we added consumptions in the execution of our skill to track a certain type of consumption referred to as `API_CALL`. Here is how to set the budget limit on our agent in the `interact` method of the `Web3CopilotAgent` class:

```python
        api_call_limit = Consumption(10, "call", "API_CALL")

        budget = Budget(
            60,
            limits=[
                api_call_limit,
            ]
        )

        result = self.agent.execute(context=self.context, budget=budget)
```

You can learn more about the budgeting [here](https://council.dev/en/latest/reference/runners/budget.html).